In [1]:
import gradio as gr

from transformers import AutoModelForCausalLM, AutoTokenizer

2021-07-18 23:45:31.083087: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-07-18 23:45:31.083131: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
model_name = "/home/shared/models/gpt-code-clippy-125M-apps-lr-adam1e-4-bs128/ckpt-1633/"
model = AutoModelForCausalLM.from_pretrained(model_name, from_flax=True)
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125M")
tokenizer.pad_token = tokenizer.eos_token

/home/arto/transformers/src/transformers/modeling_flax_pytorch_utils.py:201: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  pt_model_dict[flax_key] = torch.from_numpy(flax_tensor)
All Flax model weights were used when initializing GPTNeoForCausalLM.

Some weights of GPTNeoForCausalLM were not initialized from the Flax model and are newly initialized: ['transformer.h.4.attn.attention.bias', 'transformer.h.8.attn.attention.masked_bias', 'transformer.h.0.attn.attention.bias', 'transformer.h.2.attn.attention.masked_bias', 'transformer.h.2.attn.attention.bias', 'transformer.h.8.attn.atte

In [3]:
def format_input(question, starter_code=""):
    answer_type = "\nUse Call-Based format\n" if starter_code else "\nUse Standard Input format\n"
    return f"\nQUESTION:\n{question}\n{starter_code}\n{answer_type}\nANSWER:\n"

In [4]:
def format_outputs(text):
    formatted_text =f'''
    <head>
      <link rel="stylesheet"
            href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/10.0.3/styles/default.min.css">
      <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/10.0.3/highlight.min.js"></script>
      <script>hljs.initHighlightingOnLoad();</script>
    </head>
    <body>
      <pre><code class="python">{text}</code></pre>
    </body>
    '''
    return formatted_text

In [9]:
def generate_solution(question, starter_code="", temperature=1., num_beams=1):
    prompt = format_input(question, starter_code)
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    start = len(input_ids[0])
    output = model.generate(
        input_ids,
        max_length=start+200,
        do_sample=True,
        top_p=0.95,
        pad_token_id=tokenizer.pad_token_id,
        early_stopping=True,
        temperature=1.,
        num_beams=int(num_beams),
        no_repeat_ngram_size=None,
        repetition_penalty=None,
        num_return_sequences=None,
    )
    
    return format_outputs(tokenizer.decode(output[0][start:]).strip())

In [10]:
_EXAMPLES = [
    [
        """
Given a 2D list of size `m * n`. Your task is to find the sum of minimum value in each row.
For Example:
```python
[
  [1, 2, 3, 4, 5],       # minimum value of row is 1
  [5, 6, 7, 8, 9],       # minimum value of row is 5
  [20, 21, 34, 56, 100]  # minimum value of row is 20
]
```
So, the function should return `26` because sum of minimums is as `1 + 5 + 20 = 26`
        """,
        "",
        0.8,
    ],
    [
        """
# Personalized greeting

Create a function that gives a personalized greeting. This function takes two parameters: `name` and `owner`.
        """,
        """
Use conditionals to return the proper message:

case| return
--- | ---
name equals owner | 'Hello boss'
otherwise         | 'Hello guest'
def greet(name, owner):
        """,
        0.8,
    ]
] 

In [11]:
inputs = [
    gr.inputs.Textbox(placeholder="Define a problem here...", lines=7),
    gr.inputs.Textbox(placeholder="Provide optional starter code...", lines=3),
    gr.inputs.Slider(0.5, 1.5, 0.1, default=0.8, label="Temperature"),
    gr.inputs.Slider(1,4,1,default=1, label="Beam size")
]

outputs = [
    gr.outputs.HTML(label="Solution")
]

gr.Interface(
    generate_solution, 
    inputs=inputs, 
    outputs=outputs,
    title="Code Clippy: Problem Solver",
    examples=_EXAMPLES,
).launch(share=True)

Running locally at: http://127.0.0.1:7861/
This share link will expire in 24 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted (NEW!)
Running on External URL: https://34711.gradio.app
Interface loading below...


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7861/',
 'https://34711.gradio.app')